# 打印任务
- 一个具体的实例配置如下：
  - 一个实验室，在任意的一个小时内，大约有10名学生在场
  - 在一小时中，每人会发起2次左右打印，每次1~20页
- 打印机的性能是：
  - 以草稿模式打印的话，每分钟10页
  - 以正常模式打印的话，打印质量好，但速度下降为每分钟5页
- 问题：怎么设定打印机的模式，让大家都<font color=red>不会等太久</font>的前提下，<font color=red>尽量提高打印质量</font>?
  - 典型的决策支持问题，但无法通过规则直接计算。可以用以一段程序来<font color=red>模拟</font>这种打印任务场景，然后对程序运行结果进行<font color=red>分析</font>，以支持对打印机模式设定的<font color=red>决策</font>。
- 对问题建模
  - 对问题进行抽象，确定相关的对象过程。抛弃那些对问题实质没有关系的学生性别、年龄、、打印机型号、打印内容、纸张大小等等众多细节
  - **对象**：打印任务、打印队列、打印机
    - 打印任务的属性：提交时间、打印页数
    - 打印队列的属性：具有FIFO性质的打印任务队列
    - 打印机的属性：打印速度、是否忙
  - 过程：生成和提交打印任务
    - 确定<font color=red>生成概率</font>：实例为每小时会有10个学生提交的20个作业，这样，概率是每180秒会有1个作业生成并提交，概率为每秒1/180.
    - 确定<font color=red>打印页数</font>：实例时1~20页，那么就是1~20页之间概率相同
  - 过程：实施打印
    - 当前的打印作业：正在打印的作业
    - 打印结束倒计时：新作业开始打印时开始倒计时，回0表示打印完毕，可以处理下一个作业
  - 模拟时间：
    - 统一的时间框架：以最小单位（秒）均匀流逝的时间，设定结束时间
    - 同步所有过程：在<font color=red>一个时间单位</font>里，对<font color=red>生成打印任务</font>和<font color=red>实施打印</font>两个过程各处理一次

In [12]:
from pythonds.basic.queue import Queue
import random

class Printer:
    def __init__(self,ppm):
        self.pagerate=ppm # 打印速度
        self.currentTask=None # 打印任务
        self.timeRemaining=0 # 任务倒计时
    def tick(self):
        if self.currentTask!=None:
            self.timeRemaining=self.timeRemaining-1
            if self.timeRemaining<=0:
                self.currentTask=None
    def busy(self):
        if self.currentTask!=None:
            return True
        else:
            return False
    def startNext(self,newtask):
        self.currentTask=newtask # 这是Task类
        self.timeRemaining=newtask.getPages()*60/self.pagerate

class Task:
    def __init__(self,time):
        self.timestamp=time # 生成时间戳
        self.pages=random.randrange(1,21) # 打印页数
    def getStamp(self):
        return self.timestamp
    def getPages(self):
        return self.pages
    def waitTime(self,currenttime): # 等待时间
        return currenttime-self.timestamp 

def newPrintTask():
    num=random.randrange(1,181) # 每秒有1/180的概率生成作业
    if num==180:
        return True
    else:
        return False

def simulation(numSeconds,pagesPerMinute):
    labprinter=Printer(pagesPerMinute)
    printQueue=Queue()
    waitingtimes=[]
    for currentSecond in range(numSeconds):
        if newPrintTask():
            task=Task(currentSecond)
            printQueue.enqueue(task)
        if (not labprinter.busy()) and (not printQueue.isEmpty()):
            nexttask=printQueue.dequeue() # 出列的是task类
            waitingtimes.append(nexttask.waitTime(currentSecond))
            labprinter.startNext(nexttask)
        # 如果打印机忙
        labprinter.tick()

    averageWait=sum(waitingtimes)/len(waitingtimes)
    print('Average Wait %6.2f secs %3d tasks remaining.'%(averageWait,printQueue.size()))

In [17]:
for i in range(10):
    simulation(3600,5)

Average Wait 347.08 secs   3 tasks remaining.
Average Wait  12.67 secs   0 tasks remaining.
Average Wait 134.94 secs   0 tasks remaining.
Average Wait  52.61 secs   0 tasks remaining.
Average Wait  64.07 secs   0 tasks remaining.
Average Wait 146.69 secs   4 tasks remaining.
Average Wait  11.38 secs   0 tasks remaining.
Average Wait 122.26 secs   4 tasks remaining.
Average Wait 174.53 secs   9 tasks remaining.
Average Wait 227.92 secs   2 tasks remaining.


In [15]:
for i in range(10):
    simulation(3600,10)

Average Wait   2.38 secs   0 tasks remaining.
Average Wait  27.10 secs   1 tasks remaining.
Average Wait  12.06 secs   0 tasks remaining.
Average Wait  19.11 secs   0 tasks remaining.
Average Wait   4.39 secs   0 tasks remaining.
Average Wait  17.31 secs   0 tasks remaining.
Average Wait  11.50 secs   0 tasks remaining.
Average Wait  27.95 secs   0 tasks remaining.
Average Wait  18.00 secs   0 tasks remaining.
Average Wait  38.05 secs   0 tasks remaining.
